In [ ]:
"""
The program below takes in an image of four particles (3 plastic and 1 glass bead), 
and classifies the image based on the location of the glass bead. If it is on the short
axis the shape is 'c' for 'chevron', and if on the long axis it is 't' for 'top'.
"""

In [ ]:
"""Imports"""
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage #scikit module containing the functions for measuring image properties
import skvideo.io # reads in videos
from skimage import measure, morphology 
from skimage.filters import *
from skimage.util import invert
from scipy import ndimage
from operator import attrgetter
import numpy as np
import math
from itertools import chain
from glob import glob

In [ ]:
"""Auxiliary Functions"""


# read in a video file, display the first and last image, return the video
def full_avi(filename):
    Vid = skvideo.io.vread(filename)

    fig, ax = plt.subplots(ncols=2)
    ax[0].imshow(Vid[0]) # first fame of video
    ax[1].imshow(Vid[-1]) #last frame of video

    print("Video shape is: ", Vid.shape)
    print("Video length is: ", len(Vid))
    
    return Vid

# take a vido composed of rgb frames (3 values per pixel); make it an array composed of greyscale frames (1 value per pixel)
def greens(framelist):
    green = framelist[:, :, :, 1] #1 is the 2nd rgb channel (green channel) => take ch 2 only
    return green


# classify a raft
def sideify(idir): #idir is the list with all the properties of the thresholded image of the raft
    iner = idir.inertia_tensor[0, 0] + idir.inertia_tensor[1, 1] #moment of iertia
    maal = idir.major_axis_length 
    mial = idir.minor_axis_length
    sol = idir.solidity 
    ecc = idir.eccentricity
    cva = idir.convex_area
    area = idir.area
    if area < 1280: #too  few particles
        return "ucf"
    elif ecc <= 0.7 and  mial > 41:
        return "t"
    elif ecc > 0.8 and ecc < 0.91 and maal > 66.5 and maal < 74: # maal > 67.5 and or (mial < 41 and maal > 65): # it's on top
        return "c"
    else:
        return "ucf"



def rotate_point(point, angle):
    x0 = point[0]
    y0 = point[1]
    x1 = x0*np.cos(angle) - y0*np.sin(angle)
    y1 = x0*np.sin(angle) + y0*np.cos(angle)
    return (x1, y1)


In [ ]:
"CLASSIFIER AND FILTER"
#Many parameters for filtering are determined elsewhere, and then used in thsi notebook
# apply total thresholding to each of a list of frames; filter slides with wrong number
def total_threshold_filter(framelist, indes):
    # returns a new list, different from the orginal
    fll = len(framelist)
    f_indices = []
    f_ap = f_indices.append
    threshed_images = []
    t_ap = threshed_images.append
    m_lab = measure.label #label() is a function from measure(which I imported). This makes m_lab the name of label()
    m_rop = measure.regionprops #same. It should be faster to use m_rop multiple times than to call measure.regionprops each time to use it
    
    sides = {"t":[], "c":[], "ucf":[]} # dictionary with the potential classifications. 'ucf' means unclassified
   
    for i in range(fll):
        frame = framelist[i]
        thresh_img = frame > threshold_li(frame) # binary image
        img_labelled = m_lab(thresh_img) # contains connected regions
        properties_list = m_rop(img_labelled, coordinates = 'rc') # data about regions, for each connected region
        
        #----------getting maximum connected region----------
        biggest_r = max(properties_list, key = attrgetter('area'))
        biggest_r = m_rop(morphology.dilation(morphology.closing(m_lab(biggest_r.filled_image))), coordinates='rc')[0]

        #----------------------filter------------------------- can use actual filter
        area_threshold = 1875   # must have enough particles
        sol_threshold = 0.95
        iner_threshold = (340, 500)
        convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
        minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
        #major_thresholds = (59, 76)
        major_threshold = 78
        #inertia_thresholds = (355, 455)
        #ecc_threshold = .88
        test = (biggest_r.area > area_threshold
               #and biggest_r.major_axis_length < major_threshold # must then be well-formed
               )
        
        if test: # keep frames that have enough paricles, and are not transitions
            if (biggest_r.inertia_tensor[0, 0] + biggest_r.inertia_tensor[1,1] > iner_threshold[0]
                and biggest_r.inertia_tensor[0, 0] + biggest_r.inertia_tensor[1,1] < iner_threshold[1]):
                f_ap(i) # image's index 
                i_threshed = frame > threshold_isodata(frame)
                img2_labelled = m_lab(i_threshed)
                properties2_list = m_rop(img2_labelled)
                biggest_r2 = max(properties2_list, key = attrgetter('area'))
                t_ap((img_labelled, img2_labelled))
                #-----------------------------classify pt 1---------------------------------------
                side = sideify(biggest_r2)
                sides[side].append((frame, (img_labelled, img2_labelled), indes[i]))
            
    filtrate = framelist[f_indices]
    return [[(filtrate[i], threshed_images[i], indes[i]) for i in range(len(filtrate))], sides]




In [ ]:
# read video
filename = glob("./tchakamau/2*/*4.avi")[0]
print(filename)
frame_vid = full_avi(filename)

In [ ]:
# h is a list of indices for picking frames out of the video
h = list(range(0, len(frame_vid), int(len(frame_vid)/300)))
print(len(h))

In [ ]:
#take a sample of ~300 frames from the full video for testing 
framelist = frame_vid[h]
plt.imshow(framelist[-1]) #an example frame
print(framelist.shape)

In [ ]:
"""Pipeline for lists of frames"""
def piped(framelist, h): #h is passed because the filter retains the ability to return the indices of the frames it fileterd out and of the frames it classified, for ease of double checking
    green_frames = greens(framelist) # making a green slice
    filtered_frames = total_threshold_filter(green_frames, h)
    return filtered_frames

def isinar(a, als): #is-in-array: is a given frame included in a lst of frames
    #print(a)
    y = False
    for ele in als:
        y = np.array_equal(ele, a)
        if y:
            return y
        else: 
            continue
    return y

org = piped(framelist, h)
twf = org[0]
sids = org[1]
print("original ", len(framelist))
print("classified ", len(twf))
filts = [j[ :, :, 2] for j in framelist if not isinar(j[:, :, 2], [k[0] for k in twf] )] #jnot in [k[0] for k in twf]]
print("length of filtered out slides ", len(filts))


In [ ]:
"""Visualizethe objectz which get past the first test (area is big enough)"""
for key in sids.keys():
    vals = sids[key]
    frames = [val[0] for val in vals]
    threshes =  [val[1] for val in vals]
    c =5
    r = int(len(vals)/ c)
    for j in range(r):
        fig, ax = plt.subplots(ncols=c, figsize = (14, 10))
        for k in range(c):
            frame = frames[j * c + k]
            ax[k].set_title(key)
            ax[k].imshow(frame)
        fig, ax = plt.subplots(ncols=c, figsize = (14, 10))
        for k in range(c):
            lab = threshes[j * c + k]
            ax[k].set_title(key)
            ax[k].imshow(lab[0])
        fig, ax = plt.subplots(ncols=c, figsize = (14, 10))
        for k in range(c):
            lab = threshes[j * c + k]
            ax[k].set_title(key)
            ax[k].imshow(lab[1])
    
    

In [ ]:
"""Visualize the objects whose area is too small"""

frames = filts
c = 5
r = int(len(filts)/ c)
for j in range(r):
    fig, ax = plt.subplots(ncols=c, figsize = (14, 10))
    for k in range(c):
        frame = frames[j * c + k]
        ax[k].set_title(key)
        ax[k].imshow(frame)
    fig, ax = plt.subplots(ncols=c, figsize = (14, 10))
    for k in range(c):
        frame = frames[j * c + k]
        ax[k].set_title(key)
        ax[k].imshow(measure.label(frame > threshold_li(frame)))
    fig, ax = plt.subplots(ncols=c, figsize = (14, 10))
    for k in range(c):
        frame = frames[j * c + k]
        ax[k].set_title(key)
        ax[k].imshow(measure.label(frame > threshold_isodata(frame)))

In [ ]:
filtered_len = len(twf)
skeys = list(sids.keys())
slcs = [len(sids[x]) for x in skeys]
print("classification: ", [[skeys[i], slcs[i], '%.2f%%' %(slcs[i] *100/ filtered_len)] for i in range(len(skeys))])

In [ ]:
"How to locate the trasnparent dot itself"

In [ ]:
def region_selector(labeled_image, label):
    #print(label)
    x = labeled_image == label
    #plt.imshow(x)
    return x

# make a function that takes a set of labeled regions, and then  returns a boolean array containing only the largest
def largest_region_extractor(labeled_regions_set):
    props_lists = measure.regionprops(labeled_regions_set)
    #print(len(labeled_regions_set), len(props_lists))
    biggest_r_p = props_lists[0]
    biggest_r_label = props_lists[0].label
    for i in range(0, len(props_lists)):
        pli = props_lists[i]
        if pli.area > biggest_r_p.area:
            biggest_r_p = pli
            biggest_r_label = pli.label
        else:
            pass
    return region_selector(labeled_regions_set, biggest_r_label)
def get_transparent_dot(framelist, threshes_labeled):
    ones = []
    for i in range(len(framelist)):
        frame = framelist[i] 
        threshes = threshes_labeled[i]
        i_threshed  = threshes[1]
        y_threshed  = threshes[0]
        full = largest_region_extractor(y_threshed)
        four = largest_region_extractor(i_threshed)
        one_and_some = full^four
        ones.append(largest_region_extractor(measure.label(one_and_some)))
        
        
    return ones

In [ ]:
"""CAPTURE THE TRANSPARENT DOT ALONE FROM THE IMAGE"""
#print dots
for key in sids.keys():
    vals = sids[key]
    frames = [val[0] for val in vals]
    threshes =  [val[1] for val in vals]
    ones = get_transparent_dot(frames, threshes)
    c = 5
    r = int(len(vals)/ c)
    for j in range(r):
        fig, ax = plt.subplots(ncols=c, figsize = (14, 10))
        for k in range(c):
            frame = frames[j * c + k]
            ax[k].set_title(key)
            ax[k].imshow(frame)
        fig, ax = plt.subplots(ncols=c, figsize = (14, 10))
        for k in range(c):
            dot = ones[j * c + k]
            ax[k].set_title(key)
            ax[k].imshow(dot)

In [ ]:
#locate the dot on a grid
m_rop = measure.regionprops
morph_hull = morphology.convex_hull_image
ave = np.average
for key in sids.keys():
    vals = sids[key]
    frames = [val[0] for val in vals]
    threshes =  [val[1] for val in vals]
    ones = get_transparent_dot(frames, threshes)
    c = 5
    r = int(len(vals)/ c)
    for j in range(r):
        fig, ax = plt.subplots(ncols=c, figsize = (14, 10))
        for k in range(c):
            frame = frames[j * c + k]
            ax[k].imshow(frame)
            ax[k].set_title(key)
        plt.show()
        fig, ax = plt.subplots(ncols=c, figsize = (14, 10))
        for k in range(c):
            frame = frames[j * c + k] 
            lab = threshes[j * c + k]
            full = morphology.dilation(morphology.closing(largest_region_extractor(lab[0])))
            four = morphology.dilation(morphology.closing(largest_region_extractor(lab[1])))
            full_or = m_rop(full.astype(int), coordinates = 'rc')[0].orientation
            one_and_some = full^four
            one = largest_region_extractor(measure.label(one_and_some))
            ax[k].imshow(one)
            ax[k].set_title(key)
        plt.show()
        fig, ax = plt.subplots(ncols=c, figsize = (14, 10))
        for k in range(c):
            frame = frames[j * c + k] 
            lab = threshes[j * c + k]
            full = morphology.dilation(morphology.closing(largest_region_extractor(lab[0])))
            four = morphology.dilation(morphology.closing(largest_region_extractor(lab[1])))
            full_or = m_rop(full.astype(int), coordinates = 'rc')[0].orientation
            one_and_some = full^four
            one = largest_region_extractor(measure.label(one_and_some))
            rotated_one = ndimage.rotate(one, math.degrees(-full_or), reshape = False)
            rotated_full = ndimage.rotate(full, math.degrees(-full_or), reshape = False)
            geo_cent = m_rop(rotated_full.astype(int), coordinates = 'rc')[0].centroid#m_rop(rotated_full.astype(int), coordinates = 'rc')[0].bbox
            #print(geo_cent)

            cent_x = int(geo_cent[1])#int(ave([geo_cent[1], geo_cent[3]]) )
            cent_y = int(geo_cent[0])#int(ave([geo_cent[0], geo_cent[2]]))
            #print(cent_x, cent_y)

            one_com = m_rop(rotated_one.astype(int))[0].centroid

            rel_com_one = (one_com[1] - cent_x, one_com[0] - cent_y) # vector of com of the dot relative to geometric center
            #print("com of dot relative geo_cent", rel_com_one, np.prod(rel_com_one))
            
            #----------
            #----------
            min_vec = (1, 0)
            maj_vec = (0, 1)

            raw_coords = (m_rop(rotated_full.astype(int), coordinates = 'rc')[0].coords).T
            ys = raw_coords[0]
            #print(m_rop(rotated_full.astype(int), coordinates = 'rc')[0].coords)
            xs = raw_coords[1]
            
            scale = 10
            #plt.plot( np.nonzero(c_hull)[1]- ave(xs),  np.nonzero(c_hull)[0] - ave(ys),'y.')
            ax[k].plot(xs, ys, 'k.')
            #ax[k].plot(top.T[0], top.T[1])
            #ax[k].imshow(measure.label(morphology.dilation(top)), cmap=plt.get_cmap('hot'),)
            ax[k].plot(rel_com_one[0], rel_com_one[1], 'mo', ms=12)
            ax[k].plot([maj_vec[0]*-scale*2, maj_vec[0]*scale*2],
                     [maj_vec[1]*-scale*2, maj_vec[1]*scale*2], color='red')
            ax[k].plot([min_vec[0]*-scale, min_vec[0]*scale],
                     [min_vec[1]*-scale, min_vec[1]*scale], color='blue')
            ax[k].set_title(key)
        #plt.gca().invert_yaxis()  # Match the image system with origin at top left
        plt.show()